In [47]:


import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import numpy as np


import data_generator


In [48]:
# load in the data 

full_dataset = np.load("ml_dataset.npy");

In [49]:
# split up the data into the x and y halves

x_set = full_dataset[0];
y_set = full_dataset[1];

In [50]:
# split the data into training and testing sets


half = int(len(x_set)/2);

print(f'half: {half}');
x_train = x_set[0:half];
x_test = x_set[half:];

y_train = y_set[0:half];
y_test = y_set[half:];



half: 50


In [51]:
# print some data diagnostics to ensure we correctly obtained data sets

print(f"x_set shape: {x_set.shape}");
print(f"y_set shape: {y_set.shape}");


training_point_number = x_set.shape[1];
print(f"number of points in each graph set: {training_point_number}");


print("number of x training data sets:");
print(len(x_train[0:]));


print("number of y training data sets:");
print(len(y_train[0:]));

assert(len(x_train) == len(y_train));
assert(len(x_test) == len(y_test));
assert(x_set.shape == y_set.shape);

print(f"\nNumber of training graph-solution sets: {len(x_train)}")


print(f"Number of testing graph-solution sets: {len(x_test)}")



print(f"\nDataset shape: {x_set.shape}")



x_set shape: (100, 500)
y_set shape: (100, 500)
number of points in each graph set: 500
number of x training data sets:
50
number of y training data sets:
50

Number of training graph-solution sets: 50
Number of testing graph-solution sets: 50

Dataset shape: (100, 500)


In [52]:
# reshape the x data for the LSTM


x_train = x_train.reshape(x_train.shape[0],training_point_number,1);
x_test = x_test.reshape(x_test.shape[0],training_point_number,1);

print(x_train.shape)

(50, 500, 1)


In [59]:
#build the model by stacking layers; choose optimizer and loss function. Here we set the activation function to a 'relu'


model = tf.keras.models.Sequential()

model.add(LSTM(training_point_number));
model.add(Dense(training_point_number,activation='relu'));
model.add(Dropout(0.2));
'''
model.add(Dense(training_point_number,activation='relu'));
model.add(Dropout(0.2));

model.add(Dense(training_point_number,activation='relu'));
model.add(Dropout(0.2));
model.add(Dense(training_point_number,activation='relu'));
model.add(Dropout(0.2));
'''

model.add(Dense(training_point_number,activation='softmax'));


print("model built");


model built


In [60]:
# get the predictions for the second datapoint
predictions = model(x_train)

# print the raw predictions for that datapoint. Note that these are "logits" or "log-odds" scores, which is related to the probability, 
# and also is the inverse of the sigmoid function
print("logits prediction: ");
print(predictions[0]);

# we can use the softmax function to convert these logits to probabilities for each preciction class
print("odds prediction: ");
print(tf.nn.softmax(predictions).numpy()[0]);


print(f'shape: {predictions.shape}');


logits prediction: 
tf.Tensor(
[0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0

In [61]:
print(y_train[0])

[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf in

In [62]:
# get the loss function as a SparseCategoricalCrossentropy loss from tf.keras.losses. 
loss_fn = tf.keras.losses.MeanAbsoluteError();


print("loss:");
print(loss_fn(y_train[0],predictions[0]).numpy());

loss:
inf


In [63]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-7,decay=1e-15);

model.compile(loss=loss_fn,optimizer=opt,metrics=['accuracy']);


In [64]:
model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test), batch_size=10);

Epoch 1/10
2/2 [==============================] - 58s 6s/step - loss: inf - accuracy: 0.0600 - val_loss: 3.7170 - val_accuracy: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 9s 5s/step - loss: inf - accuracy: 0.0600 - val_loss: 3.7170 - val_accuracy: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 9s 5s/step - loss: inf - accuracy: 0.0600 - val_loss: 3.7170 - val_accuracy: 0.0000e+00
Epoch 4/10
2/2 [==============================] - ETA: 0s - loss: inf - accuracy: 0.0600

In [65]:

print(model.summary());

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 500)               1004000   
_________________________________________________________________
dense_10 (Dense)             (None, 500)               250500    
_________________________________________________________________
dropout_5 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 500)               250500    
Total params: 1,505,000
Trainable params: 1,505,000
Non-trainable params: 0
_________________________________________________________________
None
